In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wsdm-kkbox/songs.csv
/kaggle/input/wsdm-kkbox/members.csv
/kaggle/input/wsdm-kkbox/train.csv
/kaggle/input/wsdm-kkbox/test.csv
/kaggle/input/wsdm-kkbox/song_extra_info.csv


In [3]:
%matplotlib
import lightgbm as lgb
from sklearn.metrics import precision_recall_curve,roc_auc_score,classification_report,roc_curve
from tqdm import tqdm
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import missingno as msno
import gc
from sklearn.model_selection import cross_val_score,GridSearchCV,train_test_split
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.metrics import roc_curve,roc_auc_score,classification_report,mean_squared_error,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,VotingClassifier,AdaBoostClassifier

Using matplotlib backend: agg


In [4]:
#learning from
#https://www.kaggle.com/rohandx1996/recommendation-system-with-83-accuracy-lgbm

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

In [6]:
from subprocess import check_output

train = pd.read_csv('/kaggle/input/wsdm-kkbox/train.csv')
test = pd.read_csv('/kaggle/input/wsdm-kkbox/test.csv')
songs = pd.read_csv('/kaggle/input/wsdm-kkbox/songs.csv')
members = pd.read_csv('/kaggle/input/wsdm-kkbox/members.csv')


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 6 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   msno                object
 1   song_id             object
 2   source_system_tab   object
 3   source_screen_name  object
 4   source_type         object
 5   target              int64 
dtypes: int64(1), object(5)
memory usage: 337.7+ MB


In [9]:
train = pd.read_csv('/kaggle/input/wsdm-kkbox/train.csv', dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                  'source_screen_name' : 'category',
                                                  'source_type' : 'category',
                                                  'target' : np.uint8,
                                                  'song_id' : 'category'})

In [10]:
test = pd.read_csv('/kaggle/input/wsdm-kkbox/test.csv', dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                'source_screen_name' : 'category',
                                                'source_type' : 'category',
                                                'song_id' : 'category'})

In [11]:
songs = pd.read_csv('/kaggle/input/wsdm-kkbox/songs.csv',dtype={'genre_ids': 'category',
                                                  'language' : 'category',
                                                  'artist_name' : 'category',
                                                  'composer' : 'category',
                                                  'lyricist' : 'category',
                                                  'song_id' : 'category'})

In [12]:
members = pd.read_csv('/kaggle/input/wsdm-kkbox/members.csv',dtype={'city' : 'category',
                                                      'bd' : np.uint8,
                                                      'gender' : 'category',
                                                      'registered_via' : 'category'},parse_dates=['registration_init_time','expiration_date'])

In [13]:
songs_extra = pd.read_csv('/kaggle/input/wsdm-kkbox/song_extra_info.csv')

In [14]:
song_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

In [15]:
train_members = pd.merge(train, members, on='msno', how='inner')

In [16]:
train_members_songs = pd.merge(train_members, songs, on='song_id', how='outer')

In [17]:
members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))

In [18]:
members['membership_days'] = (members['expiration_date'] - members['registration_init_time']).dt.days.astype(int)

In [19]:
# categorize membership_days 
members['membership_days'] = members['membership_days']//200
members['membership_days'] = members['membership_days'].astype('category')

In [20]:
member_cols = ['msno','city','registered_via', 'registration_year', 'expiration_year', 'membership_days']

train = train.merge(members[member_cols], on='msno', how='left')
test = test.merge(members[member_cols], on='msno', how='left')

In [21]:
train

,msno,song_id,source_system_tab,source_screen_name,source_type,target,artist_name,genre_ids,song_length,language,city,registered_via,registration_year,expiration_year,membership_days
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,Bastille,359,206471.0,52.0,1,7,2012,2017,10
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,Various Artists,1259,284584.0,52.0,13,9,2011,2017,11
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,Nas,1259,225396.0,52.0,13,9,2011,2017,11
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,Soundway,1019,255512.0,-1.0,13,9,2011,2017,11
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,Brett Young,1011,187802.0,52.0,1,7,2012,2017,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,my library,Local playlist more,local-playlist,1,Dr. Dre,1259,267958.0,52.0,4,9,2004,2017,23
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,search,Search,song,0,R. City feat. Adam Levine,465,227404.0,52.0,1,7,2013,2017,7
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,search,Search,song,1,兄弟本色G.U.T.S. (姚中仁、張震嶽、頑童MJ116),465,258298.0,3.0,1,7,2013,2017,7
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,discover,Discover Genre,online-playlist,1,Brad Mehldau Trio,2122,524146.0,52.0,5,9,2015,2017,4


In [22]:
songs_extra

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001
...,...,...,...
2295966,hLnetpF6UbPg28sSfXnPE2vsdaGsLvddlXEdJR4VTIA=,Deep Breathing,PLL431720793
2295967,N+6vJ8actKQm0S3Fpf4elipTjoAo9ev28aA5FJN5e40=,In Hiding,US5UL1519827
2295968,pv35uG0ts05mWtirM/AMOWEzbHxIVart5ZzRXqKUY1c=,Il Est Ne Le Divin Enfant,PLL431502294
2295969,QSySnm8jt2Go7byY34/PxsZP6dPCins2j2cyYquNhBo=,The Exodus Song,DEPZ69316095


In [23]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return int(isrc[5:7])//5
        else:
            return int(isrc[5:7])//5
    else:
        return np.nan
#categorize song_year per 5years
#song year could be a feature
songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)
songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

In [24]:
train = train.merge(songs_extra, on = 'song_id', how = 'left')
test = test.merge(songs_extra, on = 'song_id', how = 'left')


In [25]:
train['genre_ids'] = train['genre_ids'].str.split('|').str[0]

In [26]:
train.head(5)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,artist_name,genre_ids,song_length,language,city,registered_via,registration_year,expiration_year,membership_days,song_year
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,Bastille,359,206471.0,52.0,1,7,2012,2017,10,3.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,Various Artists,1259,284584.0,52.0,13,9,2011,2017,11,19.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,Nas,1259,225396.0,52.0,13,9,2011,2017,11,1.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,Soundway,1019,255512.0,-1.0,13,9,2011,2017,11,2.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,Brett Young,1011,187802.0,52.0,1,7,2012,2017,10,3.0


In [28]:
temp_song_length = train['song_length']
temp_song_length

0          206471.0
1          284584.0
2          225396.0
3          255512.0
4          187802.0
             ...   
7377413    267958.0
7377414    227404.0
7377415    258298.0
7377416    524146.0
7377417    254026.0
Name: song_length, Length: 7377418, dtype: float64

In [29]:
train.drop('song_length', axis = 1, inplace = True)
test.drop('song_length',axis = 1 , inplace =True)

In [30]:
song_count = train.loc[:,["song_id","target"]]

# measure repeat count by played songs
song_count1 = song_count.groupby(["song_id"],as_index=False).sum().rename(columns={"target":"repeat_count"})

# count play count by songs
song_count2 = song_count.groupby(["song_id"],as_index=False).count().rename(columns = {"target":"play_count"})

In [31]:
song_count1


,song_id,repeat_count
0,+++2AEoM0d8iZTdbnAjUm35bnGKGMXdZJSv4rrWK6JQ=,0.0
1,++/ACCkEN/+VtgrJxEqeRgRmV4y8pcarDJ9T/yRAi1E=,0.0
2,++/lJNswCU+za2pYB0cWIbGL5UzWIKtfweX20+GImZA=,0.0
3,++4/NK5qpbTZWln/6UmykB8cLfRTCCj8E36IKZVzBjM=,0.0
4,++4Ihbdp0juQ9ldp9DysOL1WTLHIiawg7cnBTn55I/k=,0.0
...,...,...
359961,zzvfk6Np17ieMkvG9CQNxdDYQENCVLXuYx5VWrNhCvg=,2.0
359962,zzwCHrZc0TezilVRRTbsiWY6ORNpBq1Zv8GGtHT2xlk=,1.0
359963,zzwePOl2yF8NLVL7ZfTU2CmtlwnjSruAOpcYiod9iHs=,0.0
359964,zzxGtSW9L/V3sRNKZbkOgziHtftZ9/oNvETOTY/QHr8=,2.0


In [32]:
song_repeat = song_count1.merge(song_count2,how="inner",on="song_id")
song_repeat["repeat_percentage"] = round((song_repeat['repeat_count']*100) / song_repeat['play_count'],1)
song_repeat['repeat_count'] = song_repeat['repeat_count'].astype('int')
song_repeat['repeat_percentage'] = song_repeat['repeat_percentage'].replace(100.0,np.nan)
#cuz most of 100.0 are played=1 repeated=1 values. I think it is not fair compare with other played a lot songs

In [33]:
song_repeat.head(5)

,song_id,repeat_count,play_count,repeat_percentage
0,+++2AEoM0d8iZTdbnAjUm35bnGKGMXdZJSv4rrWK6JQ=,0,1,0.0
1,++/ACCkEN/+VtgrJxEqeRgRmV4y8pcarDJ9T/yRAi1E=,0,2,0.0
2,++/lJNswCU+za2pYB0cWIbGL5UzWIKtfweX20+GImZA=,0,3,0.0
3,++4/NK5qpbTZWln/6UmykB8cLfRTCCj8E36IKZVzBjM=,0,1,0.0
4,++4Ihbdp0juQ9ldp9DysOL1WTLHIiawg7cnBTn55I/k=,0,1,0.0


In [34]:
train = train.merge(song_repeat,on="song_id",how="left")
test = test.merge(song_repeat,on="song_id",how="left")

In [35]:
# type cast
test['song_id'] = test['song_id'].astype('category')
test['repeat_count'] = test['repeat_count'].fillna(0)
test['repeat_count'] = test['repeat_count'].astype('int')
test['play_count'] = test['play_count'].fillna(0)
test['play_count'] = test['play_count'].astype('int')

In [41]:
artist_count = train.loc[:,["artist_name","target"]]

# measure repeat count by played songs
artist_count1 = artist_count.groupby(["artist_name"],as_index=False).sum().rename(columns={"target":"repeat_count_artist"})

# measure play count by songs
artist_count2 = artist_count.groupby(["artist_name"],as_index=False).count().rename(columns = {"target":"play_count_artist"})

artist_repeat = artist_count1.merge(artist_count2,how="inner",on="artist_name")

In [44]:
artist_repeat["repeat_percentage_artist"] = round((artist_repeat['repeat_count_artist']*100) / artist_repeat['play_count_artist'],1)
artist_repeat['repeat_count_artist'] = artist_repeat['repeat_count_artist'].fillna(0)
artist_repeat['repeat_count_artist'] = artist_repeat['repeat_count_artist'].astype('int')
artist_repeat['repeat_percentage_artist'] = artist_repeat['repeat_percentage_artist'].replace(100.0,0.0)
artist_repeat['repeat_percentage_artist'] = artist_repeat['repeat_percentage_artist'].fillna(0)

In [45]:
artist_repeat

,artist_name,repeat_count_artist,play_count_artist,repeat_percentage_artist
0,!Dela Dap,0,0,0.0
1,#1 Beatles Now,7,32,21.9
2,#1 Club Hits - Best Of Dance & Techno,5,7,71.4
3,#1 Club Hits 2010 - Best Of Dance Techno (50 T...,1,4,25.0
4,#1 Country All-Star Ensemble,0,0,0.0
...,...,...,...,...
222358,시와 Siwa,0,0,0.0
222359,야콥| 레이나| 우노| 데이데이,4,7,57.1
222360,오소연,0,0,0.0
222361,장윤정| 서병순,0,1,0.0


In [46]:
del artist_repeat['repeat_count_artist']

In [47]:
train = train.merge(artist_repeat,on="artist_name",how="left")
test = test.merge(artist_repeat,on="artist_name",how="left")

In [48]:
del train['artist_name']
del test['artist_name']

In [49]:
msno_count = train.loc[:,["msno","target"]]

# count repeat count by played songs
msno_count1 = msno_count.groupby(["msno"],as_index=False).sum().rename(columns={"target":"repeat_count_msno"})

# count play count by songs
msno_count2 = msno_count.groupby(["msno"],as_index=False).count().rename(columns = {"target":"play_count_msno"})

msno_repeat = msno_count1.merge(msno_count2,how="inner",on="msno")

In [50]:
msno_repeat["repeat_percentage_msno"] = round((msno_repeat['repeat_count_msno']*100) / msno_repeat['play_count_msno'],1)
msno_repeat['repeat_count_msno'] = msno_repeat['repeat_count_msno'].fillna(0)
msno_repeat['repeat_count_msno'] = msno_repeat['repeat_count_msno'].astype('int')
#msno_repeat['repeat_percentage_msno'] = msno_repeat['repeat_percentage_msno'].replace(100.0,np.nan)
# it can be meaningful so do not erase 100.0 

In [51]:
msno_repeat

,msno,repeat_count_msno,play_count_msno,repeat_percentage_msno
0,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,293,589,49.7
1,++AH7m/EQ4iKe6wSlfO/xXAJx50p+fCeTyF90GoE9Pg=,141,220,64.1
2,++e+jsxuQ8UEnmW40od9Rq3rW7+wAum4wooXyZTKJpk=,76,108,70.4
3,++xWL5Pbi2CpG4uUugigQahauM0J/sBIRloTNPBybIU=,1,16,6.2
4,+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=,82,222,36.9
...,...,...,...,...
30750,zzTnhToV1XfDIsDJ1YaXM3Q05xV54T8Q0zTJkakXsks=,0,17,0.0
30751,zzZBJUYXrb168A4Ff4sA8L2iOH0x4ciKdD6WkV53XaE=,113,281,40.2
30752,zzompfSaMamqvjyCMYvgUBwYrxh8fHE40z3f73CQoak=,229,461,49.7
30753,zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=,113,265,42.6


In [52]:
train = train.merge(msno_repeat,on="msno",how="left")
test = test.merge(msno_repeat,on="msno",how="left")

In [57]:

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377418 entries, 0 to 7377417
Data columns (total 22 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   msno                      category
 1   song_id                   category
 2   source_system_tab         category
 3   source_screen_name        category
 4   source_type               category
 5   target                    uint8   
 6   genre_ids                 category
 7   language                  category
 8   city                      category
 9   registered_via            category
 10  registration_year         int64   
 11  expiration_year           int64   
 12  membership_days           category
 13  song_year                 float64 
 14  repeat_count              int64   
 15  play_count                int64   
 16  repeat_percentage         float64 
 17  play_count_artist         float64 
 18  repeat_percentage_artist  float64 
 19  repeat_count_msno         int64   
 20  pl

In [56]:
for col in train.columns:
    if train[col].dtype == object:
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')

In [58]:
train['song_year'] = train['song_year'].astype('category')
test['song_year'] = test['song_year'].astype('category')

In [59]:
drop_list = ['repeat_count','repeat_percentage',
             'repeat_percentage_artist',
             'repeat_count_msno','repeat_percentage_msno']
train = train.drop(drop_list,axis=1)
test = test.drop(drop_list,axis=1)

In [60]:
train

,msno,song_id,source_system_tab,source_screen_name,source_type,target,genre_ids,language,city,registered_via,registration_year,expiration_year,membership_days,song_year,play_count,play_count_artist,play_count_msno
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,359,52.0,1,7,2012,2017,10,3.0,215,1140.0,5511
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,1259,52.0,13,9,2011,2017,11,19.0,1,303616.0,622
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,1259,52.0,13,9,2011,2017,11,1.0,4,289.0,622
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,1019,-1.0,13,9,2011,2017,11,2.0,1,1.0,622
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,1011,52.0,1,7,2012,2017,10,3.0,412,427.0,5511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,my library,Local playlist more,local-playlist,1,1259,52.0,4,9,2004,2017,23,19.0,59,659.0,462
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,search,Search,song,0,465,52.0,1,7,2013,2017,7,3.0,1010,1010.0,744
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,search,Search,song,1,465,3.0,1,7,2013,2017,7,3.0,10500,25781.0,744
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,discover,Discover Genre,online-playlist,1,2122,52.0,5,9,2015,2017,4,1.0,11,32.0,601


In [61]:
test['play_count_msno'] = test['play_count_msno'].fillna(0)
test['play_count_msno'] = test['play_count_msno'].astype('int')
train['play_count_artist'] = train['play_count_artist'].fillna(0)
test['play_count_artist'] = test['play_count_artist'].fillna(0)
train['play_count_artist'] = train['play_count_artist'].astype('int')
test['play_count_artist'] = test['play_count_artist'].astype('int')

In [62]:
from sklearn.model_selection import KFold
# Create a Cross Validation with 3 splits
kf = KFold(n_splits=3)

predictions = np.zeros(shape=[len(test)])

# For each KFold
for train_indices ,validate_indices in kf.split(train) : 
    train_data = lgb.Dataset(train.drop(['target'],axis=1).loc[train_indices,:],label=train.loc[train_indices,'target'])
    val_data = lgb.Dataset(train.drop(['target'],axis=1).loc[validate_indices,:],label=train.loc[validate_indices,'target'])

    params = {
            'objective': 'binary',
            'boosting': 'gbdt',
            'learning_rate': 0.2 ,
            'verbose': 0,
            'num_leaves': 2**8,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 1,
            'feature_fraction': 0.9,
            'feature_fraction_seed': 1,
            'max_bin': 256,
            'num_rounds': 80,
            'metric' : 'auc'
        }
    # Train the model    
    lgbm_model = lgb.train(params, train_data, 100, valid_sets=[val_data])
    predictions += lgbm_model.predict(test.drop(['id'],axis=1))
    #del lgbm_model
    # We get the ammount of predictions from the prediction list, by dividing the predictions by the number of Kfolds.
predictions = predictions/3

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.733302
[2]	valid_0's auc: 0.740613
[3]	valid_0's auc: 0.747309
[4]	valid_0's auc: 0.751581
[5]	valid_0's auc: 0.761197
[6]	valid_0's auc: 0.763257
[7]	valid_0's auc: 0.764654
[8]	valid_0's auc: 0.766686
[9]	valid_0's auc: 0.767819
[10]	valid_0's auc: 0.768848
[11]	valid_0's auc: 0.769481
[12]	valid_0's auc: 0.770083
[13]	valid_0's auc: 0.770546
[14]	valid_0's auc: 0.77122
[15]	valid_0's auc: 0.771857
[16]	valid_0's auc: 0.772285
[17]	valid_0's auc: 0.772435
[18]	valid_0's auc: 0.772673
[19]	valid_0's auc: 0.772988
[20]	valid_0's auc: 0.773312
[21]	valid_0's auc: 0.773504
[22]	valid_0's auc: 0.77378
[23]	valid_0's auc: 0.773814
[24]	valid_0's auc: 0.773879
[25]	valid_0's auc: 0.773891
[26]	valid_0's auc: 0.773883
[27]	valid_0's auc: 0.773847
[28]	valid_0's auc: 0.773796
[29]	valid_0's auc: 0.773905
[30]	valid_0's auc: 0.773961
[31]	valid_0's auc: 0.773825
[32]	valid_0's auc: 0.773864
[33]	valid_0's auc: 0.774008
[34]	valid_0's auc: 0.77404
[35]	valid_0's auc: 0.7740

In [63]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from google.colab import files
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz

from surprise import SVD, SVDpp, KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate,train_test_split, GridSearchCV
from surprise import NormalPredictor
from surprise import Reader

import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
ratings_array = ratings['rating'].unique()
max_rating = np.amax( ratings_array )
min_rating = np.amin( ratings_array )
print( ratings_array )